<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Stock_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'ASIANPAINT',
       'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP',
       'DMART', 'FINEORG', 'GICRE', 'GILLETTE', 'GLAND', 'HDFCAMC',
       'ICICIGI', 'INDIGOPNTS', 'INFY', 'IRCTC', 'JMFINANCIL', 'JUBLFOOD',
       'LUXIND', 'NIACL', 'PAGEIND', 'PIDILITIND', 'PNB', 'RELAXO',
       'RELIANCE', 'TEAMLEASE', 'UJJIVANSFB', 'VAIBHAVGBL', 'VALIANTORG',
       'WIPRO'], dtype=object)

In [5]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    
    stock_df['Close'] = round(stock_df['Close'],0)     
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]   
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]   
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]   
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [6]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [7]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%'] 

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)

print('Investment: ', investment)
print('Current: ', current)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

qualified stocks: 34
Investment:  1736493.0
Current:  1784154.0
Current PnL:  47661.0
Current PnL %:  2.74
Estimate:  3254683.0
Estimate PnL:  1518190.0
Estimate PnL %:  87.43


In [8]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%
32,582.0,507.0,475.0,572.0,VALIANTORG.NS,1851.0,481.73,64.0,2T3Y,2T3Y,30830.72,37248.0,118464.0,20.81,284.24,263.43,1.78
0,357.0,315.0,300.0,311.0,5PAISA.NS,553.0,311.17,97.0,V40N,ATH,30183.49,34629.0,53641.0,14.73,77.72,62.99,1.74
17,1472.0,1331.0,1175.0,1312.0,INDIGOPNTS.NS,3112.0,1312.07,166.0,V40N,2T3Y,217803.62,244352.0,516592.0,12.19,137.18,124.99,12.54
6,6932.0,6662.0,6163.0,6601.0,BAJFINANCE.NS,7901.0,6365.23,16.0,V40,ATH,101843.68,110912.0,126416.0,8.90,24.13,15.23,5.86
7,264.0,243.0,223.0,244.0,BANDHANBNK.NS,723.0,242.80,125.0,2T3Y,2T3Y,30350.00,33000.0,90375.0,8.73,197.78,189.05,1.75
16,1191.0,1099.0,1087.0,1159.0,ICICIGI.NS,1620.0,1106.17,46.0,V40,ATH,50883.82,54786.0,74520.0,7.67,46.45,38.78,2.93
25,2583.0,2523.0,2429.0,2546.0,PIDILITIND.NS,2902.0,2442.09,20.0,V40,ATH,48841.80,51660.0,58040.0,5.77,18.83,13.06,2.81
8,1577.0,1526.0,1465.0,1643.0,BATAINDIA.NS,2173.0,1501.51,67.0,V40,ATH,100601.17,105659.0,145591.0,5.03,44.72,39.69,5.79
18,1314.0,1275.0,1319.0,1469.0,INFY.NS,1898.0,1252.12,40.0,V40,ATH,50084.80,52560.0,75920.0,4.94,51.58,46.64,2.88
33,400.0,388.0,377.0,394.0,WIPRO.NS,713.0,382.90,130.0,2T3Y,2T3Y,49777.00,52000.0,92690.0,4.47,86.21,81.74,2.87


In [9]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%
32,582.0,507.0,475.0,572.0,VALIANTORG.NS,1851.0,481.73,64.0,2T3Y,2T3Y,30830.72,37248.0,118464.0,20.81,284.24,263.43,1.78
14,912.0,1222.0,1253.0,1667.0,GLAND.NS,4316.0,1354.78,37.0,2T3Y,2T3Y,50126.86,33744.0,159692.0,-32.68,218.58,251.26,2.89
22,1466.0,1464.0,1339.0,1571.0,LUXIND.NS,4546.0,1499.87,34.0,V40N,2T3Y,50995.58,49844.0,154564.0,-2.26,203.09,205.35,2.94
31,296.0,309.0,304.0,321.0,VAIBHAVGBL.NS,966.0,320.64,96.0,2T3Y,2T3Y,30781.44,28416.0,92736.0,-7.68,201.27,208.95,1.77
23,119.0,117.0,108.0,104.0,NIACL.NS,351.0,116.60,260.0,TBD,ATH,30316.00,30940.0,91260.0,2.06,201.03,198.97,1.75
7,264.0,243.0,223.0,244.0,BANDHANBNK.NS,723.0,242.80,125.0,2T3Y,2T3Y,30350.00,33000.0,90375.0,8.73,197.78,189.05,1.75
17,1472.0,1331.0,1175.0,1312.0,INDIGOPNTS.NS,3112.0,1312.07,166.0,V40N,2T3Y,217803.62,244352.0,516592.0,12.19,137.18,124.99,12.54
29,2253.0,2186.0,2214.0,2626.0,TEAMLEASE.NS,5281.0,2239.86,14.0,V40N,2T3Y,31358.04,31542.0,73934.0,0.59,135.77,135.18,1.81
1,1390.0,1422.0,1559.0,1894.0,AAVAS.NS,3196.0,1405.00,15.0,2T3Y,2T3Y,21075.00,20850.0,47940.0,-1.07,127.47,128.54,1.21
12,178.0,173.0,154.0,147.0,GICRE.NS,358.0,171.85,175.0,2T3Y,2T3Y,30073.75,31150.0,62650.0,3.58,108.32,104.74,1.73


In [10]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%
28,2521.0,2462.0,2369.0,2472.0,RELIANCE.NS,2811.0,2464.49,21.0,V40,ATH,51754.29,52941.0,59031.0,2.29,14.06,11.77,2.98
25,2583.0,2523.0,2429.0,2546.0,PIDILITIND.NS,2902.0,2442.09,20.0,V40,ATH,48841.80,51660.0,58040.0,5.77,18.83,13.06,2.81
4,3137.0,3072.0,2926.0,3067.0,ASIANPAINT.NS,3566.0,3048.30,17.0,V40,ATH,51821.10,53329.0,60622.0,2.91,16.98,14.07,2.98
6,6932.0,6662.0,6163.0,6601.0,BAJFINANCE.NS,7901.0,6365.23,16.0,V40,ATH,101843.68,110912.0,126416.0,8.90,24.13,15.23,5.86
5,1450.0,1409.0,1344.0,1512.0,BAJAJFINSV.NS,1907.0,1449.21,35.0,V40,NaN,50722.35,50750.0,66745.0,0.05,31.59,31.54,2.92
16,1191.0,1099.0,1087.0,1159.0,ICICIGI.NS,1620.0,1106.17,46.0,V40,ATH,50883.82,54786.0,74520.0,7.67,46.45,38.78,2.93
3,427.0,408.0,390.0,457.0,AMBUJACEM.NS,588.0,408.85,120.0,TBD,ATH,49062.00,51240.0,70560.0,4.44,43.82,39.38,2.83
24,37896.0,41158.0,39390.0,43996.0,PAGEIND.NS,53631.0,39840.50,2.0,V40,ATH,79681.00,75792.0,107262.0,-4.88,34.61,39.49,4.59
8,1577.0,1526.0,1465.0,1643.0,BATAINDIA.NS,2173.0,1501.51,67.0,V40,ATH,100601.17,105659.0,145591.0,5.03,44.72,39.69,5.79
9,4119.0,4139.0,4094.0,4552.0,BAYERCROP.NS,6002.0,4125.00,7.0,V40N,NaN,28875.00,28833.0,42014.0,-0.15,45.50,45.65,1.66


In [11]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%
17,1472.0,1331.0,1175.0,1312.0,INDIGOPNTS.NS,3112.0,1312.07,166.0,V40N,2T3Y,217803.62,244352.0,516592.0,12.19,137.18,124.99,12.54
6,6932.0,6662.0,6163.0,6601.0,BAJFINANCE.NS,7901.0,6365.23,16.0,V40,ATH,101843.68,110912.0,126416.0,8.90,24.13,15.23,5.86
8,1577.0,1526.0,1465.0,1643.0,BATAINDIA.NS,2173.0,1501.51,67.0,V40,ATH,100601.17,105659.0,145591.0,5.03,44.72,39.69,5.79
24,37896.0,41158.0,39390.0,43996.0,PAGEIND.NS,53631.0,39840.50,2.0,V40,ATH,79681.00,75792.0,107262.0,-4.88,34.61,39.49,4.59
13,4408.0,4488.0,4402.0,4879.0,GILLETTE.NS,7529.0,4479.47,14.0,V40,ATH,62712.58,61712.0,105406.0,-1.60,68.08,69.68,3.61
4,3137.0,3072.0,2926.0,3067.0,ASIANPAINT.NS,3566.0,3048.30,17.0,V40,ATH,51821.10,53329.0,60622.0,2.91,16.98,14.07,2.98
28,2521.0,2462.0,2369.0,2472.0,RELIANCE.NS,2811.0,2464.49,21.0,V40,ATH,51754.29,52941.0,59031.0,2.29,14.06,11.77,2.98
10,3503.0,3528.0,3471.0,3880.0,DMART.NS,5324.0,3702.36,14.0,SS,ATH,51833.04,49042.0,74536.0,-5.38,43.80,49.18,2.98
22,1466.0,1464.0,1339.0,1571.0,LUXIND.NS,4546.0,1499.87,34.0,V40N,2T3Y,50995.58,49844.0,154564.0,-2.26,203.09,205.35,2.94
21,482.0,471.0,451.0,521.0,JUBLFOOD.NS,903.0,468.21,109.0,SS,ATH,51034.89,52538.0,98427.0,2.95,92.86,89.91,2.94


In [12]:
tmp_df.to_csv('/content/drive/My Drive/data/stocks/Stock-Portfolio-Analysis.csv', index=False)

In [13]:
tmp_df.sort_values(by = 'Stock', ascending=True)

,Close,20_SMA,50_SMA,200_SMA,Stock,Max,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%
0,357.0,315.0,300.0,311.0,5PAISA.NS,553.0,311.17,97.0,V40N,ATH,30183.49,34629.0,53641.0,14.73,77.72,62.99,1.74
1,1390.0,1422.0,1559.0,1894.0,AAVAS.NS,3196.0,1405.00,15.0,2T3Y,2T3Y,21075.00,20850.0,47940.0,-1.07,127.47,128.54,1.21
2,604.0,620.0,597.0,573.0,AMARAJABAT.NS,988.0,626.83,48.0,V200,ATH,30087.84,28992.0,47424.0,-3.64,57.62,61.26,1.73
3,427.0,408.0,390.0,457.0,AMBUJACEM.NS,588.0,408.85,120.0,TBD,ATH,49062.00,51240.0,70560.0,4.44,43.82,39.38,2.83
4,3137.0,3072.0,2926.0,3067.0,ASIANPAINT.NS,3566.0,3048.30,17.0,V40,ATH,51821.10,53329.0,60622.0,2.91,16.98,14.07,2.98
5,1450.0,1409.0,1344.0,1512.0,BAJAJFINSV.NS,1907.0,1449.21,35.0,V40,NaN,50722.35,50750.0,66745.0,0.05,31.59,31.54,2.92
6,6932.0,6662.0,6163.0,6601.0,BAJFINANCE.NS,7901.0,6365.23,16.0,V40,ATH,101843.68,110912.0,126416.0,8.90,24.13,15.23,5.86
7,264.0,243.0,223.0,244.0,BANDHANBNK.NS,723.0,242.80,125.0,2T3Y,2T3Y,30350.00,33000.0,90375.0,8.73,197.78,189.05,1.75
8,1577.0,1526.0,1465.0,1643.0,BATAINDIA.NS,2173.0,1501.51,67.0,V40,ATH,100601.17,105659.0,145591.0,5.03,44.72,39.69,5.79
9,4119.0,4139.0,4094.0,4552.0,BAYERCROP.NS,6002.0,4125.00,7.0,V40N,NaN,28875.00,28833.0,42014.0,-0.15,45.50,45.65,1.66
